In [2]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.special import expit, logit

In [3]:
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
init_notebook_mode(connected=True)

In [430]:
number_of_samples = 500

x = np.linspace(0,10,number_of_samples).reshape(number_of_samples,1)

X = np.concatenate((np.power(x,4),np.power(x,3),np.power(x,2),
                    x,
                    np.ones(x.shape)),axis=1)

A = np.array([1,0,0,2,10]).reshape(-1,1)

epsilon = 500
y = np.matmul(X,A) + epsilon*np.random.randn(number_of_samples,1) 
data = np.concatenate((X,y),axis=1)

number_of_itterations=2000

In [431]:
data.shape

(500, 6)

In [432]:
X.shape

(500, 5)

In [433]:
y.shape

(500, 1)

In [434]:
iplot(go.Figure(data=[go.Scatter(x=x.ravel(),y=y.ravel())]))

In [435]:
def cost(y,y_pred):
    return np.power(y_pred-y,2).sum()

In [436]:
def ols_derivative(X,y,W):
    derivative = -np.matmul(X.T,y-np.matmul(X,W))/len(y)
    return derivative

In [437]:
def ridge_derivative(X,y,W,beta=1):
    derivative = ols_derivative(X,y,W)+beta*W/len(y)
    return derivative

In [438]:
def lasso_derivative(X,y,W,gamma=1):
    derivative = ols_derivative(X,y,W)+gamma/len(y)
    return derivative

In [439]:
def hesian(X):
    return np.matmul(X.T,X)

In [440]:
def optimal_learning_rate(X,y,W):
    grad = ols_derivative(X,y,W)
    return np.matmul(grad.T,grad)/np.matmul(np.matmul(grad.T,hesian(X)),grad)

In [441]:
def gradient_step(X,y,W, derivative_function):
    return W - optimal_learning_rate(X,y,W)*derivative_function(X,y,W)

In [442]:
def run_regression(X, y, iterations=10000):
    W = np.random.randn(X.shape[1],1)
    for i in range(iterations):
        W = gradient_step(X,y,W,ols_derivative)
        y_pred = np.matmul(X,W)
        if i%(iterations/20) ==0:
            print ("iteration:", i, "MSE:", cost(y,y_pred))
    return W

In [443]:
W = run_regression(X, y) 

iteration: 0 MSE: 200622954.529
iteration: 500 MSE: 132064726.121
iteration: 1000 MSE: 122804230.425
iteration: 1500 MSE: 121552754.345
iteration: 2000 MSE: 121383008.244
iteration: 2500 MSE: 121359328.33
iteration: 3000 MSE: 121354699.005
iteration: 3500 MSE: 120398624.515
iteration: 4000 MSE: 119866727.831
iteration: 4500 MSE: 119618152.725
iteration: 5000 MSE: 119506839.218
iteration: 5500 MSE: 119450143.823
iteration: 6000 MSE: 119422990.477
iteration: 6500 MSE: 119410939.777
iteration: 7000 MSE: 119404653.136
iteration: 7500 MSE: 119401185.373
iteration: 8000 MSE: 119399213.791
iteration: 8500 MSE: 119397871.091
iteration: 9000 MSE: 119396959.009
iteration: 9500 MSE: 119396246.963


In [444]:
iplot(go.Figure(data=[go.Scatter(x=x.ravel(),y=np.matmul(X,W).ravel(),name='predicted'),
                      go.Scatter(x=x.ravel(),y=np.matmul(X,A).ravel(),name='original'),
                      go.Scatter(x=x.ravel(),y=y.ravel(),name='noisy')]))